In [1]:
# ==============================================================================

# 👩‍💻 Author    : Hyelim Jo
# 🎯 Purpose   : AI 윤리성 리스크 진단 에이전트 v1.0
# 📅 Created   : 2025-10-22
# 📜 Note      : service_profiler.ipynb

# ==============================================================================

In [2]:
# -------------------------------- Update Log ----------------------------------

# 2025-10-22 14:00 / 초기 생성 / Service Profiler 기본 구조 구현
# 2025-10-22 15:30 / API 키 오류 해결 / load_dotenv() 추가

# ------------------------------------------------------------------------------

In [3]:
# step1. 라이브러리 불러오기
from typing import Dict, Any
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

print("[OK] 라이브러리 불러오기 완료")

[OK] 라이브러리 불러오기 완료


In [4]:

# step2. 프롬프트 템플릿 정의 (키 이름 변경: data_handling_method, user_impact_scope, diagnosed_risk_categories)
prompt_template = """당신은 AI 윤리 전문가입니다.

주어진 AI 서비스 설명을 분석하여 다음 정보를 JSON 형식으로 추출해주세요.

# 서비스 설명:
{service_description}

# 추출할 정보:
1. service_name: 서비스 이름
2. service_type: 서비스 유형 (recommendation, classification, prediction 등 가장 적절한 단어 선택)
3. description: 핵심 설명 (1-2문장)
4. data_handling_method: 처리하는 데이터 및 처리 방식 (최종 보고서의 '데이터 처리 방식' 항목에 해당)
5. user_impact_scope: 사용자 영향 범위 (최종 보고서의 '사용자 영향 범위' 항목에 해당)
6. diagnosed_risk_categories: 리스크 카테고리 (bias, privacy, transparency, accountability, safety, security 중 선택, 배열)

JSON 형식으로만 답변해주세요.
"""

prompt = PromptTemplate.from_template(prompt_template)

print("[OK] 프롬프트 정의 완료")



[OK] 프롬프트 정의 완료


In [5]:
# step3. LLM 및 Chain 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
json_parser = JsonOutputParser()
chain = prompt | llm | json_parser

print("[OK] Chain 구성 완료")


[OK] Chain 구성 완료


In [6]:
# step4. Service Profiler 함수 정의
def service_profiler(state: Dict[str, Any]) -> Dict[str, Any]:
    """서비스 프로파일 생성"""
    service_description = state.get("service_description", "")
    
    # 💡 LLM 호출 시도
    try:
        result = chain.invoke({"service_description": service_description})
    except Exception as e:
        print(f"[ERROR] LLM 호출 실패: {e}")
        # 오류 시 기본 구조 반환
        result = {
            "service_name": "N/A",
            "service_type": "N/A",
            "description": "LLM 호출 실패로 프로파일 생성 불가",
            "data_handling_method": "N/A",
            "user_impact_scope": "N/A",
            "diagnosed_risk_categories": []
        }

    # State에 결과 저장
    state["service_profile"] = result
    
    print(f"[OK] 프로파일 생성 완료 - {result.get('service_name')} ({result.get('service_type')})")
    return state

print("[OK] 함수 정의 완료")


[OK] 함수 정의 완료


In [7]:
# step5. 테스트
test_state = {
    "service_description": "채용 지원자의 이력서를 AI로 분석하여 적합한 후보자를 추천하는 시스템입니다. 학력, 경력, 자격증을 바탕으로 직무 적합도를 평가합니다. 개인 식별 정보는 익명화하여 사용합니다.",
    "service_profile": {}
}

result_state = service_profiler(test_state)

print("\n[결과]")
print("서비스명:", result_state["service_profile"].get("service_name"))
print("서비스 유형:", result_state["service_profile"].get("service_type"))
print("데이터 처리 방식:", result_state["service_profile"].get("data_handling_method")) # 변경된 키
print("사용자 영향 범위:", result_state["service_profile"].get("user_impact_scope"))     # 변경된 키
print("리스크 카테고리:", ", ".join(result_state["service_profile"].get("diagnosed_risk_categories", []))) # 변경된 키
print("\n전체 프로파일:")
print(result_state["service_profile"])


[OK] 프로파일 생성 완료 - 이력서 분석 추천 시스템 (recommendation)

[결과]
서비스명: 이력서 분석 추천 시스템
서비스 유형: recommendation
데이터 처리 방식: 개인 식별 정보는 익명화하여 사용합니다.
사용자 영향 범위: 채용 과정에서 지원자와 기업 모두에게 영향을 미칩니다.
리스크 카테고리: bias, privacy, transparency

전체 프로파일:
{'service_name': '이력서 분석 추천 시스템', 'service_type': 'recommendation', 'description': '채용 지원자의 이력서를 AI로 분석하여 적합한 후보자를 추천하는 시스템입니다. 학력, 경력, 자격증을 바탕으로 직무 적합도를 평가합니다.', 'data_handling_method': '개인 식별 정보는 익명화하여 사용합니다.', 'user_impact_scope': '채용 과정에서 지원자와 기업 모두에게 영향을 미칩니다.', 'diagnosed_risk_categories': ['bias', 'privacy', 'transparency']}


In [ ]:
# step6. State 기반 실행 함수 정의
import sys
sys.path.append('..')
from state_manager import load_state, save_state, update_status

def service_profiler_execute(service_description: str):
    """Service Profiler 실행 함수"""
    print("\n" + "="*60)
    print("🔍 Service Profiler 시작...")
    print("="*60)
    
    # State 로드
    state = load_state()
    
    # 서비스 설명 설정
    state["service_description"] = service_description
    
    # 서비스 프로파일 생성
    result = chain.invoke({"service_description": service_description})
    state["service_profile"] = result
    
    # State 저장
    save_state(state)
    update_status(state, "service_profiler", "completed")
    
    print(f"✅ 프로파일 생성 완료 - {result.get('service_name')} ({result.get('service_type')})")
    print(f"✅ 리스크 카테고리: {', '.join(result.get('risk_categories', []))}")
    
    return state

print("✅ Service Profiler 실행 함수 정의 완료")
